<a href="https://colab.research.google.com/github/chirkser/Generative-adversarial-networks/blob/master/CopulaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [1]:
! pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.0 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 61 kB 316 kB/s 
     |████████████████████████████████| 1.6 MB 26.7 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 131 kB 54.1 MB/s 
     |████████████████████████████████| 11.2 MB 29.5 MB/s 
     |████████████████████████████████| 965 kB 46.8 MB/s 
     |████████████████████████████████| 280 kB 58.1 MB/s 
     |████████████████████████████████| 636 kB 57.9 MB/s 
     |████████████████████████████████| 15.3 MB 24.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successf

In [2]:
import pandas as pd
from sdv.tabular import CopulaGAN
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from sdmetrics import load_demo
from sdmetrics.reports.single_table import QualityReport


# Preparing data

In [3]:
from sdv.demo import load_tabular_demo
original_data = load_tabular_demo('student_placements')
original_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


# Training model

In [4]:
model =CopulaGAN(primary_key='student_id')
model.fit(original_data)
synthetic_data = model.sample(1000)
synthetic_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,65.04,53.85,Science,79.15,Sci&Tech,False,0,68.09,Mkt&Fin,67.94,26788.0,True,NaT,2020-08-11,NaN
1,1,M,44.84,44.93,Commerce,77.84,Sci&Tech,False,0,85.98,Mkt&HR,67.78,32601.0,False,NaT,NaT,NaN
2,2,F,80.07,57.49,Commerce,80.59,Comm&Mgmt,False,2,97.92,Mkt&Fin,77.72,29484.0,True,NaT,2020-06-23,5.0
3,3,M,60.33,52.49,Commerce,67.92,Comm&Mgmt,True,0,73.85,Mkt&HR,74.90,27335.0,True,2020-02-11,2020-06-17,NaN
4,4,M,47.08,37.74,Science,90.00,Others,True,0,93.25,Mkt&Fin,75.27,NaN,True,2020-02-19,2020-03-21,NaN


# Evaluation


In [5]:
original_data, synthetic_data, metadata = load_demo(modality='single_table')

my_report = QualityReport()
my_report.generate(original_data, synthetic_data, metadata)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.66it/s]



Overall Quality Score: 82.84%

Properties:
Column Shapes: 82.78%
Column Pair Trends: 82.9%


In [6]:
my_report.get_visualization(property_name='Column Pair Trends')